### Task

1. Add new column with values `Yes/No` to whether `Country` is `Mexico`.
2. Group by new column and sum `bytes_used`.
3. Group by Country and use sqlfunc CountDistinct to calculate the number of IP addresses seen in each country.

In [1]:
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-2.3.1/spark-2.3.1-bin-hadoop2.7.tgz
!tar xf spark-2.3.1-bin-hadoop2.7.tgz
!pip install -q findspark

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.3.1-bin-hadoop2.7"

!ls

import findspark
findspark.init()

import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate() 
spark

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease [15.9 kB]
Hit:14 

In [2]:
import numpy as np
from pyspark.sql import functions as f
from pyspark.sql import types

In [5]:
schema = types.StructType([
                           types.StructField("idip_address", types.StringType()),
                           types.StructField("Country", types.StringType()),
                           types.StructField("Domain Name", types.StringType()),
                           types.StructField("Bytes_used", types.IntegerType()),
])

In [6]:
df = spark.read.csv("challenge.csv", schema=schema, header=True)
df.show(5)

+--------------+---------+----------------+----------+
|  idip_address|  Country|     Domain Name|Bytes_used|
+--------------+---------+----------------+----------+
| 52.81.192.172|    China|odnoklassniki.ru|       463|
|119.239.207.13|    China|        youtu.be|        51|
| 68.69.217.210|    China|       adobe.com|        10|
|  7.191.21.223| Bulgaria|    linkedin.com|       853|
|  211.13.10.68|Indonesia|         hud.gov|        29|
+--------------+---------+----------------+----------+
only showing top 5 rows



In [7]:
df_new = df.withColumn('is_Mexico',
                       f.when(df.Country == 'Mexico', 'Yes').otherwise('No')
 )

In [12]:
df_new.show(20)

+---------------+--------------+-----------------+----------+---------+
|   idip_address|       Country|      Domain Name|Bytes_used|is_Mexico|
+---------------+--------------+-----------------+----------+---------+
|  52.81.192.172|         China| odnoklassniki.ru|       463|       No|
| 119.239.207.13|         China|         youtu.be|        51|       No|
|  68.69.217.210|         China|        adobe.com|        10|       No|
|   7.191.21.223|      Bulgaria|     linkedin.com|       853|       No|
|   211.13.10.68|     Indonesia|          hud.gov|        29|       No|
|   239.80.21.97|      Suriname|       smh.com.au|       218|       No|
|106.214.106.233|       Jamaica|    amazonaws.com|        95|       No|
| 127.242.24.138|         China| surveymonkey.com|       123|       No|
|     99.2.6.139|Czech Republic|     geocities.jp|       322|       No|
|   237.54.11.63|         China|       amazon.com|        83|       No|
| 252.141.157.25|         Japan|      cornell.edu|       374|   

In [13]:
df_new_grouped = df_new.groupBy('is_Mexico').agg(f.sum('Bytes_used').alias('Sum_bytes'))
df_new_grouped.show()

+---------+---------+
|is_Mexico|Sum_bytes|
+---------+---------+
|       No|   508076|
|      Yes|     6293|
+---------+---------+



In [26]:
df_grouped = df.groupBy('Country').agg(f.countDistinct('idip_address').alias('num_of_unique_ip'))
df_grouped.show(10)

+-----------+----------------+
|    Country|num_of_unique_ip|
+-----------+----------------+
|       Chad|               1|
|     Russia|              56|
|   Paraguay|               1|
|      Yemen|               1|
|     Sweden|              28|
|Philippines|              65|
|   Malaysia|               5|
|     Turkey|               1|
|     Malawi|               2|
|    Germany|               5|
+-----------+----------------+
only showing top 10 rows



In [25]:
df_grouped_sorted = df_grouped.sort(f.col('num_of_unique_ip').desc())
df_grouped_sorted.show(10)

+--------------+----------------+
|       Country|num_of_unique_ip|
+--------------+----------------+
|         China|             172|
|     Indonesia|             114|
|   Philippines|              65|
|        Russia|              56|
|        Brazil|              35|
|        Poland|              31|
|        Sweden|              28|
|         Japan|              25|
|Czech Republic|              23|
|      Portugal|              23|
+--------------+----------------+
only showing top 10 rows

